# Random Hyperplanes Experiment
1. Generate data according to Gaussian XOR limited to a unit cube
2. Randomly generate hyperplanes until polytopes are pure (contain only samples of the same class)
3. Assign class labels to all polytopes
4. Compute train error, generalization error, and matrix norm

## Functions

In [1]:
import numpy as np
from scipy.stats import mode
import pandas as pd
import seaborn as sns

In [2]:
def sample_gaussian_xor(n, cov_scale=1, angle_params=None, k=1, acorn=None, shuffled=True):
    means = [[-1, -1], [1, 1], [1, -1], [-1, 1]]
    blob = np.concatenate(
        [
            np.random.multivariate_normal(
                mean, cov_scale * np.eye(len(mean)), size=int(n / 4)
            )
            for mean in means
        ]
    )

    X = np.zeros_like(blob)
    Y = np.concatenate([np.ones((int(n / 4))) * int(i < 2) for i in range(len(means))])
    X[:, 0] = blob[:, 0] * np.cos(angle_params * np.pi / 180) + blob[:, 1] * np.sin(
        angle_params * np.pi / 180
    )
    X[:, 1] = -blob[:, 0] * np.sin(angle_params * np.pi / 180) + blob[:, 1] * np.cos(
        angle_params * np.pi / 180
    )
    
    if shuffled:
        idx = np.arange(X.shape[0])
        np.random.shuffle(idx)
        X = X[idx]
        Y = Y[idx]

    return X, Y.astype(int)

def sample_clipped_xor(n, cov_scale=1, angle_params=0, k=1, acorn=None, clip_bound=3, shuffled=True):
    """
    Gaussian XOR.
    """
    
    X, Y = sample_gaussian_xor(n, cov_scale, angle_params, k, acorn, shuffled)
    
    interior = np.where(
        (X[:, 0] < clip_bound) & (X[:, 0] > -clip_bound) &
        (X[:, 1] < clip_bound) & (X[:, 1] > -clip_bound))
    X = X[interior]
    Y = Y[interior]
    
    while X.shape[0] < n:
        X2, Y2 = sample_gaussian_xor(n, cov_scale, angle_params, k, shuffled=True)
        X = np.vstack((X, X2[:n - X.shape[0]]))
        Y = np.concatenate((Y, Y2[:n - Y.shape[0]]))

    return X, Y.astype(int)
    
def sample_unit_vector(dim):
    vec = np.random.normal(0, 1, (dim))
    vec /= np.linalg.norm(vec)
    return vec

In [3]:
class RandomHyperplanes:
    def __init__(self, random_labels=False, n_hyperplanes=None, random_state=None):
        """
        If random_labels, then all polytope labels are random.
        
        """
        self.random_labels = random_labels
        self.n_hyperplanes = n_hyperplanes
        self.random_state = random_state
    
    def _random_unit_vector(self):
        vec = np.random.normal(0, 1, (self.dimension_))
        vec /= np.linalg.norm(vec)
        return vec
    
    def _assign_regions(self, X):
        """
        Converts samples to their numeric polytope encoding.
        Too large to encode with powers of 2
        """
        regions = (np.sign(X @ self.hyperplanes_.T) + 1) // 2
        regions = np.asarray([''.join(bools) for bools in regions.astype(str)])
        # bool_codes = np.asarray([2**d for d in range(regions.shape[1])])
        # regions = (regions @ bool_codes).astype(int)
        return regions
    
    def _get_impure_samples(self, X, y):
        """Find the subset of samples in hetergeneous polytopes"""
        regions = self._assign_regions(X)
        impure_indices = np.empty((0), dtype=int)
        for region in np.unique(regions):
            idx = np.arange(X.shape[0])[np.where(regions == region)[0]]
            if len(set(y[idx])) > 1:
                impure_indices = np.append(impure_indices, idx)
        
        return impure_indices
    
    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.dimension_ = X.shape[1]
        self.n_classes_ = len(set(y))
        
        if self.n_hyperplanes is not None:
            self.hyperplanes_ = np.vstack([
                self._random_unit_vector() for _ in range(self.n_hyperplanes)
            ])
        else:
            self.hyperplanes_ = np.empty((0, self.dimension_), dtype=float)
            impure_indices = np.arange(X.shape[0])
        
            while impure_indices.shape[0] > 0:
                self.hyperplanes_ = np.append(self.hyperplanes_, self._random_unit_vector().reshape(1, -1), axis=0)
                subset_indices = self._get_impure_samples(X[impure_indices], y[impure_indices])
                if len(subset_indices) == 0:
                    break
                impure_indices = impure_indices[subset_indices]
        
        if self.random_labels:
            self.label_dict_ = {
                code: np.random.choice(self.n_classes_) for code in self._assign_regions(X)
            }
        else:
            self.label_dict_ = {}
            regions = self._assign_regions(X)
            for region in np.unique(regions):
                idx = np.arange(X.shape[0])[np.where(regions == region)[0]]
                self.label_dict_[region] = mode(y[idx])[0][0]
                
        return self

    def predict(self, X):
        """If a region was unoccupied by a training sample, the label is generated randomly"""
        region_codes = self._assign_regions(X)
        
        # Keep track of how many test samples are in empty regions
        self.n_empty_samples_ = 0
        for region, count in zip(*np.unique(region_codes, return_counts=True)):
            if region not in self.label_dict_.keys():
                self.n_empty_samples_ += count

        y = []
        for code in region_codes:
            try:
                y.append(self.label_dict_[code])
            except KeyError:
                random_label = np.random.choice(self.n_classes_)
                self.label_dict_[code] = random_label
                y.append(random_label)
            
        return np.asarray(y)

In [4]:
def matrix_norm(rh, X, p=2):
    str_codes = rh._assign_regions(X)
    unique, counts = np.unique(str_codes, return_counts=True)
    saturation = len(unique) / X.shape[0]
    
    return saturation, np.sum(counts**(p/2))**(1/p)

## Experiments

In [5]:
X_test, y_test = sample_clipped_xor(n=1000, acorn=1234)

In [6]:
n_runs = 10
results_mat_random = []
results_mat_labeled = []
columns = [
    'Run', 'n_hyperplanes', 'saturation', 'schatten_2norm', 'train_error', 'test_error',
    'proprotion_test_empty', 'expected_empty_cell_error']

for run in range(n_runs):
    print(f'Run {run}')
    # Takes too long to run over 60 samples. Maybe need smaller features space or different distribution
    X_train, y_train = sample_clipped_xor(n=40, acorn=run)

    # Random labels
    rh = RandomHyperplanes(random_labels=True)
    rh = rh.fit(X_train, y_train)
    y_train_hat = rh.predict(X_train)
    y_test_hat = rh.predict(X_test)
    
    train_error = np.mean(np.abs(y_train - y_train_hat))
    test_error = np.mean(np.abs(y_test - y_test_hat))
    sat, schatten2norm = matrix_norm(rh, X_train)
    fraction_samples_empty = rh.n_empty_samples_ / X_test.shape[0]
    expected_empty_error = 0.5 * fraction_samples_empty
    
    results_mat_random.append([
        run, rh.hyperplanes_.shape[0], sat, schatten2norm, train_error, test_error,
         fraction_samples_empty, expected_empty_error
    ])
    
    # Data driven labels
    rh = RandomHyperplanes(random_labels=False)
    rh = rh.fit(X_train, y_train)
    y_train_hat = rh.predict(X_train)
    y_test_hat = rh.predict(X_test)
    
    train_error = np.mean(np.abs(y_train - y_train_hat))
    test_error = np.mean(np.abs(y_test - y_test_hat))
    sat, schatten2norm = matrix_norm(rh, X_train)
    fraction_samples_empty = rh.n_empty_samples_ / X_test.shape[0]
    expected_empty_error = 0.5 * fraction_samples_empty
    
    results_mat_labeled.append([
        run, rh.hyperplanes_.shape[0], sat, schatten2norm, train_error, test_error,
         fraction_samples_empty, expected_empty_error
    ])
    
df_random_labels = pd.DataFrame(results_mat_random, columns=columns)
df_data_labels = pd.DataFrame(results_mat_labeled, columns=columns)

### Random label results

In [7]:
df_random_labels

,Run,n_hyperplanes,saturation,schatten_2norm,train_error,test_error,proprotion_test_empty,expected_empty_cell_error
0,0,5389,1.000,6.324555,0.300,0.488,0.994,0.4970
1,1,273,0.950,6.324555,0.650,0.508,0.863,0.4315
2,2,247,0.950,6.324555,0.450,0.481,0.843,0.4215
3,3,1303,1.000,6.324555,0.550,0.504,0.975,0.4875
4,4,287,1.000,6.324555,0.525,0.503,0.859,0.4295
5,5,782,1.000,6.324555,0.575,0.464,0.953,0.4765
6,6,90,0.850,6.324555,0.450,0.524,0.631,0.3155
7,7,270,0.925,6.324555,0.675,0.523,0.889,0.4445
8,8,152,0.975,6.324555,0.550,0.506,0.735,0.3675
9,9,1145,1.000,6.324555,0.550,0.521,0.962,0.4810


### Data-driven label results

In [8]:
df_data_labels

,Run,n_hyperplanes,saturation,schatten_2norm,train_error,test_error,proprotion_test_empty,expected_empty_cell_error
0,0,13511,1.000,6.324555,0.0,0.495,1.000,0.5000
1,1,623,1.000,6.324555,0.0,0.477,0.924,0.4620
2,2,311,0.925,6.324555,0.0,0.508,0.906,0.4530
3,3,386,1.000,6.324555,0.0,0.488,0.901,0.4505
4,4,384,1.000,6.324555,0.0,0.499,0.910,0.4550
5,5,166,0.950,6.324555,0.0,0.484,0.784,0.3920
6,6,283,0.950,6.324555,0.0,0.469,0.831,0.4155
7,7,74,0.825,6.324555,0.0,0.416,0.642,0.3210
8,8,4691,1.000,6.324555,0.0,0.493,0.988,0.4940
9,9,1544,1.000,6.324555,0.0,0.463,0.975,0.4875
